In [1]:
# Imports
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
epochs = 10
batch_size = 32

test_valid_size = 128

n_classes = 10
dropout = 0.75

In [3]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
def conv2d(x, W, B, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, B)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x, 
        ksize=[1, k, k, 1], 
        strides=[1, k, k, 1], 
        padding='SAME'
    )

In [6]:
def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out

In [7]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [10]:
# Model
logits = conv_net(x, weights, biases, keep_prob)

In [11]:
# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
# Initialize
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 633677.2500 Validation Accuracy: 0.148438
Epoch  1, Batch   2 -Loss: 106996.1250 Validation Accuracy: 0.101562
Epoch  1, Batch   3 -Loss:  5089.9580 Validation Accuracy: 0.078125
Epoch  1, Batch   4 -Loss:  3271.1079 Validation Accuracy: 0.085938
Epoch  1, Batch   5 -Loss:  1629.7284 Validation Accuracy: 0.046875
Epoch  1, Batch   6 -Loss:   851.1344 Validation Accuracy: 0.101562
Epoch  1, Batch   7 -Loss:   479.4319 Validation Accuracy: 0.140625
Epoch  1, Batch   8 -Loss:   232.0882 Validation Accuracy: 0.101562
Epoch  1, Batch   9 -Loss:   378.2611 Validation Accuracy: 0.140625
Epoch  1, Batch  10 -Loss:   170.0843 Validation Accuracy: 0.101562
Epoch  1, Batch  11 -Loss:   112.1624 Validation Accuracy: 0.101562
Epoch  1, Batch  12 -Loss:   134.9606 Validation Accuracy: 0.101562
Epoch  1, Batch  13 -Loss:   136.2169 Validation Accuracy: 0.101562
Epoch  1, Batch  14 -Loss:   123.5135 Validation Accuracy: 0.101562
Epoch  1, Batch  15 -Loss:    95.0964 Validati

Epoch  1, Batch 122 -Loss:    78.5822 Validation Accuracy: 0.101562
Epoch  1, Batch 123 -Loss:   116.8477 Validation Accuracy: 0.101562
Epoch  1, Batch 124 -Loss:    92.9035 Validation Accuracy: 0.101562
Epoch  1, Batch 125 -Loss:   109.3513 Validation Accuracy: 0.101562
Epoch  1, Batch 126 -Loss:   123.1759 Validation Accuracy: 0.101562
Epoch  1, Batch 127 -Loss:    94.2609 Validation Accuracy: 0.101562
Epoch  1, Batch 128 -Loss:   113.8953 Validation Accuracy: 0.101562
Epoch  1, Batch 129 -Loss:   116.2055 Validation Accuracy: 0.101562
Epoch  1, Batch 130 -Loss:    70.4395 Validation Accuracy: 0.101562
Epoch  1, Batch 131 -Loss:   102.1962 Validation Accuracy: 0.101562
Epoch  1, Batch 132 -Loss:   136.7619 Validation Accuracy: 0.101562
Epoch  1, Batch 133 -Loss:   141.7890 Validation Accuracy: 0.101562
Epoch  1, Batch 134 -Loss:    91.2373 Validation Accuracy: 0.101562
Epoch  1, Batch 135 -Loss:    96.0295 Validation Accuracy: 0.101562
Epoch  1, Batch 136 -Loss:    78.2731 Validation

Epoch  1, Batch 243 -Loss:    71.2591 Validation Accuracy: 0.101562
Epoch  1, Batch 244 -Loss:    62.8781 Validation Accuracy: 0.101562
Epoch  1, Batch 245 -Loss:    60.8459 Validation Accuracy: 0.101562
Epoch  1, Batch 246 -Loss:   111.8492 Validation Accuracy: 0.101562
Epoch  1, Batch 247 -Loss:    67.9706 Validation Accuracy: 0.101562
Epoch  1, Batch 248 -Loss:    78.2194 Validation Accuracy: 0.101562
Epoch  1, Batch 249 -Loss:    92.0703 Validation Accuracy: 0.101562
Epoch  1, Batch 250 -Loss:    60.7360 Validation Accuracy: 0.101562
Epoch  1, Batch 251 -Loss:    99.4875 Validation Accuracy: 0.101562
Epoch  1, Batch 252 -Loss:    83.8741 Validation Accuracy: 0.101562
Epoch  1, Batch 253 -Loss:    86.7898 Validation Accuracy: 0.101562
Epoch  1, Batch 254 -Loss:    49.6227 Validation Accuracy: 0.101562
Epoch  1, Batch 255 -Loss:    73.8009 Validation Accuracy: 0.101562
Epoch  1, Batch 256 -Loss:    66.6074 Validation Accuracy: 0.101562
Epoch  1, Batch 257 -Loss:    65.3742 Validation

Epoch  1, Batch 364 -Loss:    47.0111 Validation Accuracy: 0.101562
Epoch  1, Batch 365 -Loss:    95.4142 Validation Accuracy: 0.101562
Epoch  1, Batch 366 -Loss:    66.5438 Validation Accuracy: 0.101562
Epoch  1, Batch 367 -Loss:    72.1145 Validation Accuracy: 0.101562
Epoch  1, Batch 368 -Loss:    84.0052 Validation Accuracy: 0.101562
Epoch  1, Batch 369 -Loss:    80.7017 Validation Accuracy: 0.101562
Epoch  1, Batch 370 -Loss:    68.0432 Validation Accuracy: 0.101562
Epoch  1, Batch 371 -Loss:    42.7001 Validation Accuracy: 0.101562
Epoch  1, Batch 372 -Loss:    99.1632 Validation Accuracy: 0.101562
Epoch  1, Batch 373 -Loss:    40.5156 Validation Accuracy: 0.101562
Epoch  1, Batch 374 -Loss:    67.7818 Validation Accuracy: 0.101562
Epoch  1, Batch 375 -Loss:    43.5331 Validation Accuracy: 0.101562
Epoch  1, Batch 376 -Loss:    76.0922 Validation Accuracy: 0.101562
Epoch  1, Batch 377 -Loss:    57.4887 Validation Accuracy: 0.101562
Epoch  1, Batch 378 -Loss:    84.2274 Validation

Epoch  1, Batch 485 -Loss:    66.5261 Validation Accuracy: 0.101562
Epoch  1, Batch 486 -Loss:    55.0293 Validation Accuracy: 0.101562
Epoch  1, Batch 487 -Loss:    34.7607 Validation Accuracy: 0.101562
Epoch  1, Batch 488 -Loss:    36.8286 Validation Accuracy: 0.101562
Epoch  1, Batch 489 -Loss:    37.3418 Validation Accuracy: 0.101562
Epoch  1, Batch 490 -Loss:    39.2216 Validation Accuracy: 0.101562
Epoch  1, Batch 491 -Loss:    38.3013 Validation Accuracy: 0.101562
Epoch  1, Batch 492 -Loss:    42.6888 Validation Accuracy: 0.101562
Epoch  1, Batch 493 -Loss:    62.1099 Validation Accuracy: 0.101562
Epoch  1, Batch 494 -Loss:    49.4324 Validation Accuracy: 0.101562
Epoch  1, Batch 495 -Loss:    48.8665 Validation Accuracy: 0.101562
Epoch  1, Batch 496 -Loss:    29.1909 Validation Accuracy: 0.101562
Epoch  1, Batch 497 -Loss:    43.4144 Validation Accuracy: 0.101562
Epoch  1, Batch 498 -Loss:    47.3864 Validation Accuracy: 0.101562
Epoch  1, Batch 499 -Loss:    55.8861 Validation

...

Epoch  4, Batch 619 -Loss:     2.9949 Validation Accuracy: 0.156250
Epoch  4, Batch 620 -Loss:     2.8739 Validation Accuracy: 0.156250
Epoch  4, Batch 621 -Loss:     2.8232 Validation Accuracy: 0.156250
Epoch  4, Batch 622 -Loss:     2.7184 Validation Accuracy: 0.156250
Epoch  4, Batch 623 -Loss:     3.1982 Validation Accuracy: 0.156250
Epoch  4, Batch 624 -Loss:     3.0617 Validation Accuracy: 0.156250
Epoch  4, Batch 625 -Loss:     2.6380 Validation Accuracy: 0.156250
Epoch  4, Batch 626 -Loss:     2.7911 Validation Accuracy: 0.156250
Epoch  4, Batch 627 -Loss:     2.3528 Validation Accuracy: 0.156250
Epoch  4, Batch 628 -Loss:     3.2789 Validation Accuracy: 0.156250
Epoch  4, Batch 629 -Loss:     3.2051 Validation Accuracy: 0.156250
Epoch  4, Batch 630 -Loss:     2.6975 Validation Accuracy: 0.156250
Epoch  4, Batch 631 -Loss:     3.1949 Validation Accuracy: 0.156250
Epoch  4, Batch 632 -Loss:     2.8090 Validation Accuracy: 0.156250
Epoch  4, Batch 633 -Loss:     2.7537 Validation

Epoch  4, Batch 740 -Loss:     3.2708 Validation Accuracy: 0.156250
Epoch  4, Batch 741 -Loss:     2.8477 Validation Accuracy: 0.156250
Epoch  4, Batch 742 -Loss:     3.7619 Validation Accuracy: 0.156250
Epoch  4, Batch 743 -Loss:     3.1087 Validation Accuracy: 0.156250
Epoch  4, Batch 744 -Loss:     3.0868 Validation Accuracy: 0.156250
Epoch  4, Batch 745 -Loss:     3.0940 Validation Accuracy: 0.156250
Epoch  4, Batch 746 -Loss:     3.3423 Validation Accuracy: 0.156250
Epoch  4, Batch 747 -Loss:     3.1821 Validation Accuracy: 0.156250
Epoch  4, Batch 748 -Loss:     2.7652 Validation Accuracy: 0.156250
Epoch  4, Batch 749 -Loss:     2.5096 Validation Accuracy: 0.156250
Epoch  4, Batch 750 -Loss:     3.1284 Validation Accuracy: 0.156250
Epoch  4, Batch 751 -Loss:     3.1392 Validation Accuracy: 0.156250
Epoch  4, Batch 752 -Loss:     2.9795 Validation Accuracy: 0.156250
Epoch  4, Batch 753 -Loss:     2.9318 Validation Accuracy: 0.156250
Epoch  4, Batch 754 -Loss:     2.3406 Validation

Epoch  4, Batch 861 -Loss:     2.7110 Validation Accuracy: 0.156250
Epoch  4, Batch 862 -Loss:     2.6431 Validation Accuracy: 0.156250
Epoch  4, Batch 863 -Loss:     2.8953 Validation Accuracy: 0.156250
Epoch  4, Batch 864 -Loss:     2.6831 Validation Accuracy: 0.156250
Epoch  4, Batch 865 -Loss:     2.6672 Validation Accuracy: 0.156250
Epoch  4, Batch 866 -Loss:     2.7262 Validation Accuracy: 0.156250
Epoch  4, Batch 867 -Loss:     2.9168 Validation Accuracy: 0.156250
Epoch  4, Batch 868 -Loss:     2.5062 Validation Accuracy: 0.156250
Epoch  4, Batch 869 -Loss:     2.5981 Validation Accuracy: 0.156250
Epoch  4, Batch 870 -Loss:     2.4998 Validation Accuracy: 0.156250
Epoch  4, Batch 871 -Loss:     2.9774 Validation Accuracy: 0.156250
Epoch  4, Batch 872 -Loss:     2.8105 Validation Accuracy: 0.156250
Epoch  4, Batch 873 -Loss:     2.9984 Validation Accuracy: 0.156250
Epoch  4, Batch 874 -Loss:     2.6336 Validation Accuracy: 0.156250
Epoch  4, Batch 875 -Loss:     2.6345 Validation

Epoch  4, Batch 982 -Loss:     2.7906 Validation Accuracy: 0.156250
Epoch  4, Batch 983 -Loss:     3.1007 Validation Accuracy: 0.156250
Epoch  4, Batch 984 -Loss:     2.4405 Validation Accuracy: 0.156250
Epoch  4, Batch 985 -Loss:     3.0996 Validation Accuracy: 0.156250
Epoch  4, Batch 986 -Loss:     3.1533 Validation Accuracy: 0.156250
Epoch  4, Batch 987 -Loss:     2.9863 Validation Accuracy: 0.156250
Epoch  4, Batch 988 -Loss:     2.9060 Validation Accuracy: 0.156250
Epoch  4, Batch 989 -Loss:     3.4097 Validation Accuracy: 0.156250
Epoch  4, Batch 990 -Loss:     2.7736 Validation Accuracy: 0.156250
Epoch  4, Batch 991 -Loss:     2.9463 Validation Accuracy: 0.156250
Epoch  4, Batch 992 -Loss:     2.6396 Validation Accuracy: 0.156250
Epoch  4, Batch 993 -Loss:     2.7646 Validation Accuracy: 0.156250
Epoch  4, Batch 994 -Loss:     2.5532 Validation Accuracy: 0.156250
Epoch  4, Batch 995 -Loss:     3.0093 Validation Accuracy: 0.156250
Epoch  4, Batch 996 -Loss:     2.6778 Validation

Epoch  4, Batch 1101 -Loss:     2.4841 Validation Accuracy: 0.156250
Epoch  4, Batch 1102 -Loss:     2.4448 Validation Accuracy: 0.156250
Epoch  4, Batch 1103 -Loss:     2.9696 Validation Accuracy: 0.156250
Epoch  4, Batch 1104 -Loss:     2.8860 Validation Accuracy: 0.156250
Epoch  4, Batch 1105 -Loss:     2.7452 Validation Accuracy: 0.156250
Epoch  4, Batch 1106 -Loss:     2.5768 Validation Accuracy: 0.156250
Epoch  4, Batch 1107 -Loss:     2.8827 Validation Accuracy: 0.156250
Epoch  4, Batch 1108 -Loss:     2.5641 Validation Accuracy: 0.156250
Epoch  4, Batch 1109 -Loss:     2.4936 Validation Accuracy: 0.156250
Epoch  4, Batch 1110 -Loss:     2.7807 Validation Accuracy: 0.156250
Epoch  4, Batch 1111 -Loss:     2.6478 Validation Accuracy: 0.156250
Epoch  4, Batch 1112 -Loss:     2.9102 Validation Accuracy: 0.156250
Epoch  4, Batch 1113 -Loss:     2.6862 Validation Accuracy: 0.156250
Epoch  4, Batch 1114 -Loss:     2.9398 Validation Accuracy: 0.156250
Epoch  4, Batch 1115 -Loss:     3.